In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime, timedelta



In [4]:
sales_data = pd.read_csv('sales_data.txt')
products_data = pd.read_csv('products_data.txt')

sales_data['order_date'] = pd.to_datetime(sales_data['order_date'])

In [6]:
#В каком регионе было продано наибольшее количество продуктов категории "Смартфоны"?

merged_data = sales_data.merge(products_data, on='product_id')

smartphones_data = merged_data[merged_data['category'] == 'Смартфоны']

region_with_most_smartphone_sales = smartphones_data.groupby('region')['quantity'].sum().idxmax()
region_with_most_smartphone_sales

'Восток'

In [9]:
#Какова разница между максимальной и минимальной средней ценой продуктов категории "Ноутбуки" между всеми производителями? Ответ округлите до целых.
merged_data = sales_data.merge(products_data, on='product_id')

laptops_data = merged_data[merged_data['category'] == 'Ноутбуки']

manufacturer_avg_price = laptops_data.groupby('manufacturer')['price_per_unit'].mean()
round(max(manufacturer_avg_price) - min(manufacturer_avg_price))

200

In [10]:
#Какой покупатель совершил заказы на наибольшую сумму по всем временам?
customer_total_spending = sales_data.groupby('customer_id')['total_price'].sum()

top_spending_customer = customer_total_spending.idxmax()
top_spending_customer

1003

In [12]:
#Какова доля продаж продуктов категории "Планшеты" от общей выручки за все время? Ответ округлите до 2 знака после запятой. Разделитель разряда - точка.
merged_data = sales_data.merge(products_data, on='product_id')

tablets_data = merged_data[merged_data['category'] == 'Планшеты']

tablets_revenue_ratio = tablets_data['total_price'].sum() / merged_data['total_price'].sum()
tablets_revenue_ratio

0.24848484848484848

In [14]:
#Какова разница в средней цене продуктов категории "Смартфоны" между двумя самыми крупными регионами по продажам? Ответ округлите до целых.
merged_data = sales_data.merge(products_data, on='product_id')

region_revenue = merged_data.groupby('region')['total_price'].sum()

top_2_regions = region_revenue.nlargest(2)

top_regions_smartphones = merged_data[(merged_data['region'].isin(top_2_regions.index)) & (merged_data['category'] == 'Смартфоны')]

avg_price_by_region = top_regions_smartphones.groupby('region')['price_per_unit'].mean()

price_difference = avg_price_by_region.diff().abs().iloc[-1]
price_difference

275.0

In [16]:
#Сколько в среднем уникальных категорий продуктов было продано по всем регионам?
merged_data = sales_data.merge(products_data, on='product_id')

unique_categories_per_region = merged_data.groupby('region')['category'].nunique()
unique_categories_per_region.mean()

5.0

In [18]:
#Какой производитель представлен в наибольшем количестве различных категорий продуктов?
merged_data = sales_data.merge(products_data, on='product_id')

manufacturer_unique_categories = merged_data.groupby('manufacturer')['category'].nunique()

manufacturer_with_most_unique_categories = manufacturer_unique_categories.idxmax()
manufacturer_with_most_unique_categories

'E'

In [20]:
#В каком месяце было продано больше всего продуктов категории "Планшеты" в регионе "Север"?
merged_data = sales_data.merge(products_data, on='product_id')

north_tablets_data = merged_data[(merged_data['category'] == 'Планшеты') & (merged_data['region'] == 'Север')]

north_tablets_data['month'] = north_tablets_data['order_date'].dt.month

month_with_most_north_tablet_sales = north_tablets_data.groupby('month')['quantity'].sum().idxmax()
month_with_most_north_tablet_sales

8

In [22]:
#Какой производитель продал продукты на наибольшую сумму в регионе "Юг" за март 2023 года?
merged_data = sales_data.merge(products_data, on='product_id')

south_march_2023_data = merged_data[(merged_data['region'] == 'Юг') & (merged_data['order_date'].dt.month == 3) & (merged_data['order_date'].dt.year == 2023)]

manufacturer_revenue_south_march_2023 = south_march_2023_data.groupby('manufacturer')['total_price'].sum()

manufacturer_with_highest_revenue = manufacturer_revenue_south_march_2023.idxmax()
manufacturer_with_highest_revenue

'B'

In [23]:
#Сколько покупателей совершили заказы на продукты хотя бы трех разных категорий?
merged_data = sales_data.merge(products_data, on='product_id')

customer_unique_categories = merged_data.groupby('customer_id')['category'].nunique()

customers_with_multiple_categories = customer_unique_categories[customer_unique_categories >= 3]

num_customers_with_multiple_categories = len(customers_with_multiple_categories)
num_customers_with_multiple_categories

1

In [26]:
#Какова доля продаж продуктов производителя "C" от общей выручки за все время в регионе "Север"? Ответ округлите до 2 знака после запятой. В качестве разделителя разрядов используйте точку.
merged_data = sales_data.merge(products_data, on='product_id')

manufacturer_c_north_data = merged_data[(merged_data['manufacturer'] == 'C') & (merged_data['region'] == 'Север')]

revenue_ratio_manufacturer_c_north = manufacturer_c_north_data['total_price'].sum() / merged_data[merged_data['region'] == 'Север']['total_price'].sum()
revenue_ratio_manufacturer_c_north

0.26811594202898553

In [27]:
#Найдите покупателей, которые купили у более чем 15% уникальных производителей. Если таких покупателей несколько - напишите их через запятую с пробелом в порядке возрастания.
#Примечание: Уникальными производителями считайте уникальную связку "Категория = Производитель".

merged_data = sales_data.merge(products_data, on='product_id')

total_manufactures = int(merged_data.groupby(['category', 'manufacturer'])['manufacturer'].agg({'nunique'}).sum())

unique_manufacturers_per_customer = merged_data.groupby(['customer_id', 'category', 'manufacturer'])['manufacturer'].agg({'nunique'})

percent_from_total = unique_manufacturers_per_customer.groupby('customer_id').sum().apply(lambda x: x / total_manufactures).reset_index()

customers_with_more_than_15 = percent_from_total[percent_from_total['nunique'] > 0.15]

customers_with_more_than_15['customer_id'].tolist()

[1001, 1003]

In [29]:
# Какое максимальное количество продуктов было продано в один заказ?
max_quantity_per_order = sales_data.groupby('order_id')['quantity'].sum().max()
max_quantity_per_order

4

In [32]:
# Какой производитель имеет самую высокую среднюю цену продукта среди всех категорий?
merged_data = sales_data.merge(products_data, on='product_id')

average_price_per_manufacturer_per_category = merged_data.groupby(['manufacturer', 'category'])['price_per_unit'].mean()

manufacturer_with_highest_average_price = average_price_per_manufacturer_per_category.groupby('manufacturer').mean().idxmax()
manufacturer_with_highest_average_price


'E'

In [33]:
# Каково общее количество уникальных производителей, чьи продукты были куплены менее чем в 3 регионах?
merged_data = sales_data.merge(products_data, on='product_id')

unique_regions_per_manufacturer = merged_data.groupby(['manufacturer', 'region'])['region'].nunique()

manufacturers_with_products_in_multiple_regions = unique_regions_per_manufacturer.groupby('manufacturer').apply(lambda x: (len(x) <= 2))

total_manufacturers_with_products_in_multiple_regions = manufacturers_with_products_in_multiple_regions.sum()
total_manufacturers_with_products_in_multiple_regions

0